In [8]:
import xarray
import xarray_beam as xbeam

source_dataset, source_chunks = xbeam.open_zarr('gcs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2')

In [10]:
import pandas as pd
start_date = pd.to_datetime('2016-12-31')
end_date = pd.to_datetime('2021-12-31')
source_dataset = source_dataset.sel(time=slice(start_date, end_date))

In [16]:
print(source_dataset.latitude.size)

721


In [1]:
import sys,os
import torch

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd()))))))
from datasets.weather_bench import WeatherDataset
from models.VariableEncoder.datasets.dataset import CustomDataset


air = ["geopotential", 'specific_humidity', 'temperature', 'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity']
sur = ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure', 'total_cloud_cover', 'total_precipitation']
only_input_variable = ['toa_incident_solar_radiation']
constant_variable = ['geopotential_at_surface', 'land_sea_mask']

device = ("cuda" if torch.cuda.is_available() else "cpu" )
device = torch.device(device)

weather = WeatherDataset(0, device=device, offline=True)
print(weather.load(air, sur, only_input_variable=only_input_variable, constant_variables=constant_variable)[0].shape)

데이터셋 불러오는 중...
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                       (time: 87673, latitude: 30, longitude: 30,
                                   level: 37)
Coordinates:
  * latitude                      (latitude) float32 32.0 32.25 ... 39.0 39.25
  * level                         (level) int64 1 2 3 5 7 ... 925 950 975 1000
  * longitude                     (longitude) float32 124.0 124.2 ... 131.2
  * time                          (time) datetime64[ns] 2011-12-31 ... 2021-1...
Data variables: (12/15)
    10m_u_component_of_wind       (time, latitude, longitude) float32 ...
    10m_v_component_of_wind       (time, latitude, longitude) float32 ...
    2m_temperature                (time, latitude, longitude) float32 ...
    geopotential                  (time, level, latitude, longitude) float32 ...
    geopotential_at_surface       (latitude, longitude) float32 ...
    land_sea_mask                 (latitude, longitude) float32 ...
    ...                       

Processing futures: 13it [00:45,  3.48s/it]


torch.Size([2, 1, 900])
57.60417 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                       (time: 87673, latitude: 20, longitude: 20,
                                   level: 37)
Coordinates:
  * latitude                      (latitude) float64 21.0 22.5 ... 48.0 49.5
  * level                         (level) int64 1 2 3 5 7 ... 925 950 975 1000
  * longitude                     (longitude) float64 115.5 117.0 ... 144.0
  * time                          (time) datetime64[ns] 2011-12-31 ... 2021-1...
Data variables: (12/15)
    10m_u_component_of_wind       (time, latitude, longitude) float32 ...
    10m_v_component_of_wind       (time, latitude, longitude) float32 ...
    2m_temperature                (time, latitude, longitude) float32 ...
    geopotential                  (time, level, latitude, longitude) float32 ...
    geopotential_at_surface       (latitude, longitude) float32 ...
    land_sea_mask                 (latitude, longitude) float32 ...
    ...   

Processing futures: 13it [00:17,  1.37s/it]


torch.Size([2, 1, 400])
24.08766 sec
torch.Size([87673, 231, 1300])


In [11]:
def denormalize(inputs, mean_std):
    # min_max 텐서를 적절히 재구성하여 inputs의 차원에 맞춤
    mean = mean_std[:, 0].view(1, 1, mean_std.size(0), 1)
    std = mean_std[:, 1].view(1, 1, mean_std.size(0), 1)
    # 역정규화 수행
    denormalized = (inputs * std) + mean
    return denormalized

In [15]:
src, tgt, label = dataset[0]

reversed_predict = denormalize(tgt.unsqueeze(0), mean_std)

In [22]:
print(reversed_predict[0, : , :, :label.size(-1)])

tensor([[[2.0130e+05, 2.0128e+05, 2.0127e+05,  ..., 1.9970e+05,
          1.9969e+05, 1.9969e+05],
         [1.6036e+05, 1.6036e+05, 1.6036e+05,  ..., 1.5738e+05,
          1.5740e+05, 1.5741e+05],
         [1.3633e+05, 1.3634e+05, 1.3634e+05,  ..., 1.3227e+05,
          1.3227e+05, 1.3226e+05],
         ...,
         [1.0308e+05, 1.0309e+05, 1.0310e+05,  ..., 1.0268e+05,
          1.0268e+05, 1.0269e+05],
         [1.9595e-01, 1.4695e-01, 1.1039e-01,  ..., 7.8946e-01,
          8.8265e-01, 9.0048e-01],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.8327e-04,
          2.0125e-04, 1.4473e-04]],

        [[2.0138e+05, 2.0138e+05, 2.0137e+05,  ..., 1.9979e+05,
          1.9979e+05, 1.9978e+05],
         [1.6041e+05, 1.6041e+05, 1.6041e+05,  ..., 1.5743e+05,
          1.5745e+05, 1.5746e+05],
         [1.3637e+05, 1.3638e+05, 1.3638e+05,  ..., 1.3231e+05,
          1.3231e+05, 1.3231e+05],
         ...,
         [1.0310e+05, 1.0311e+05, 1.0312e+05,  ..., 1.0274e+05,
          1.027

In [21]:
print(label)

tensor([[[2.0130e+05, 2.0128e+05, 2.0127e+05,  ..., 1.9970e+05,
          1.9969e+05, 1.9969e+05],
         [1.6036e+05, 1.6036e+05, 1.6036e+05,  ..., 1.5738e+05,
          1.5740e+05, 1.5741e+05],
         [1.3633e+05, 1.3634e+05, 1.3634e+05,  ..., 1.3227e+05,
          1.3227e+05, 1.3226e+05],
         ...,
         [1.0308e+05, 1.0309e+05, 1.0310e+05,  ..., 1.0268e+05,
          1.0268e+05, 1.0269e+05],
         [1.9595e-01, 1.4695e-01, 1.1039e-01,  ..., 7.8946e-01,
          8.8265e-01, 9.0048e-01],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.8327e-04,
          2.0125e-04, 1.4473e-04]],

        [[2.0138e+05, 2.0138e+05, 2.0137e+05,  ..., 1.9979e+05,
          1.9979e+05, 1.9978e+05],
         [1.6041e+05, 1.6041e+05, 1.6041e+05,  ..., 1.5743e+05,
          1.5745e+05, 1.5746e+05],
         [1.3637e+05, 1.3638e+05, 1.3638e+05,  ..., 1.3231e+05,
          1.3231e+05, 1.3231e+05],
         ...,
         [1.0310e+05, 1.0311e+05, 1.0312e+05,  ..., 1.0274e+05,
          1.027